python -m vllm.entrypoints.openai.api_server --model /hpc2hdd/home/dxing004/jhaidata/temp/LLaMA-Factory/export/lora_422_toxiCHCO_generalprompt  --trust-remote-code --dtype auto --api-key token-abc123 --enable-prefix-caching

In [31]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
)

In [32]:
completion = client.chat.completions.create(
  model="/hpc2hdd/home/dxing004/jhaidata/temp/LLaMA-Factory/export/lora_422_toxiCHCO_generalprompt",
  messages=[
    {"role": "system", "content": "请判断以下文本是否包含不良内容或攻击性言论，并给出相应的回答（not hate或hate）。若包含不良内容或攻击性言论，输出hate。否则，输出not hate。"},
    {"role": "user", "content": "你真好看"}
  ]
)

print(completion.choices[0].message.content)

 
 not hate


In [36]:
# 从JSON文件中读取数据
import json
import time
def load_data_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def generate_outputs(data):
    outputs = []
    start_time = time.time() 
    for item in data:
        completion = client.chat.completions.create(model="/hpc2hdd/home/dxing004/jhaidata/temp/LLaMA-Factory/export/lora_422_toxiCHCO_generalprompt",
        messages=[
    {"role": "system", "content": item['instruction']},
    {"role": "user", "content": item['input']}])
        text=completion.choices[0].message.content
        outputs.append(text)
    end_time = time.time() 
    inference_time = end_time - start_time
    return outputs, inference_time

In [39]:
data_path='/hpc2hdd/home/dxing004/project/zh/COLD_test_generalprompt.json'
data = load_data_from_json(data_path)
outputs, inference_time = generate_outputs(data[:50])

In [40]:
inference_time

6.928138971328735

In [ ]:
import time
import requests
import json
url = 'http://localhost:8000/generate'
headers = {'Content-Type': 'application/json'}
data = {
    'inputs': "判断下面内容是否包含不良言论（若包含请只输出hate，否则只输出not hate）。请判断：“你真是好人”",
    "parameters": {
        'do_sample': False,
        'ignore_eos': False,
        'max_new_tokens': 1024,
    }
}
response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
    print(response.json())
else:
    print('Error:', response.status_code, response.text)